In [31]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc

import re
from nltk.corpus import stopwords
import distance
from bs4 import BeautifulSoup
import scipy
import re
    

In [2]:
df = pd.read_csv("train.csv")

In [17]:
contractions_dict = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he shall",
"he'll've": "he shall have",
"he's": "he has",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I had",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you shall have",
"you're": "you are",
"you've": "you have"}


In [19]:
def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    #expand contractions
    
    def expand_contractions(sentence, contractions_dict=contractions_dict):
        def replace(match):
            return contractions_dict[match.group(0)]
        return contractions_re.sub(replace, sentence)

    
    #print "Removing Punctuations and numbers and expanding contractions..."
    pattern = re.compile('\W')
    contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
        x = ''.join([i for i in x if not i.isdigit()])
        x = expand_contractions(x)
    
        
    #print "Removing HTML tags and performing stemming..."
    
    if type(x) == type(''):
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    
    return x
    

In [20]:
df["question1"] = df["question1"].fillna("").apply(preprocess)
df["question2"] = df["question2"].fillna("").apply(preprocess)

/home/darshan_excellence/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/darshan_excellence/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")




In [22]:
df.shape
original_shape = pd.read_csv('train.csv').shape
print df.shape,original_shape

(404290, 6) (404290, 6)


In [23]:
import nltk
bigrm1 = set(nltk.bigrams("Hey what is up?".split()))
bigrm2 = set(nltk.bigrams("Hey what do up is up?".split()))
print bigrm1,bigrm2
len(bigrm1&bigrm2)

set([('Hey', 'what'), ('is', 'up?'), ('what', 'is')]) set([('do', 'up'), ('Hey', 'what'), ('is', 'up?'), ('what', 'do'), ('up', 'is')])


2

In [24]:
#test
test = df.loc[:5]

In [26]:
def bigram_word_Common(row):
    w1 = set(nltk.bigrams(row['question1'].split(" ")))
    w2 = set(nltk.bigrams(row['question2'].split(" ")))
    return 1.0 * len(w1 & w2)

df['bigram_common'] = df.apply(bigram_word_Common,axis = 1)

def bigram_word_total(row):
    w1 = set(nltk.bigrams(row['question1'].split(" ")))
    w2 = set(nltk.bigrams(row['question2'].split(" ")))
    return 1.0 * ((len(w1)+len(w2)) - 1.0 * len(w1 & w2))

df['bigram_total'] = df.apply(bigram_word_total,axis = 1)

df['bigram_ratio'] = df['bigram_common']/df['bigram_total']

In [27]:
test['combined'] = test['question1']+" "+test['question2']
test['combined'].loc[2]

/home/darshan_excellence/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



u'how can i increase the speed of my internet connection while using a vpn  how can internet speed be increased by hacking through dns '

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
df['combined'] = df['question1']+" "+df['question2']

tfidf_combined = v.fit_transform(df['combined'].values)
word_vec = np.array(v.get_feature_names())

tfidf_rows = [tfidf_combined[i,:] for i in range(0,df.shape[0])]
df['tfidf'] = tfidf_rows

In [32]:
def tfidf_unigram_word_match(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    

    common_words = [words for words in (w1 & w2) if words != '' and len(words)>1] 
    if common_words == []:
        return 0

    else:
        tfidf_scores = scipy.sparse.find(row['tfidf'])
        common_word_ind = [np.where(word_vec == ind)[0][0] for ind in common_words]
        tfidf_common_sum = sum([tfidf_scores[2][i] for i in range(0,len(tfidf_scores[1])) if tfidf_scores[1][i] in common_word_ind])
        return 1.0*(tfidf_common_sum/sum(tfidf_scores[2]))

    
df['tfidf_unigram_word_match'] = df.apply(tfidf_unigram_word_match,axis=1)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,bigram_common,bigram_total,bigram_ratio,combined,tfidf,tfidf_unigram_word_match
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,11.0,15.0,0.733333,what is the step by step guide to invest in sh...,"(0, 77556)\t0.0756699844518\n (0, 35701)\t0...",0.966742
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,6.0,23.0,0.260870,what is the story of kohinoor koh i noor dia...,"(0, 77556)\t0.0701848729964\n (0, 35701)\t0...",0.650728
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,1.0,23.0,0.043478,how can i increase the speed of my internet co...,"(0, 70831)\t0.0540481138459\n (0, 10064)\t0...",0.301877
3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math math is di...,0,0.0,23.0,0.000000,why am i mentally very lonely how can i solve...,"(0, 35701)\t0.0668356636523\n (0, 70831)\t0...",0.000000
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.0,22.0,0.000000,which one dissolve in water quikly sugar salt...,"(0, 33839)\t0.116120577565\n (0, 78429)\t0....",0.293379


In [36]:
#here
from functools import reduce
def normalized_word_Common(row):

    w1 = set(map(lambda word: word_vec.value().index("daiict") , Common_words))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return  w1 & w2
    
    map(lambda word: word.lower().strip(), row['question2'].split(" "))
    sum = reduce((lambda x, y: x + y), )

In [38]:
#test
def normalized_word_Common(row):

    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * len(w1 & w2)

df['word_Common'] = df.apply(normalized_word_Common, axis=1)

def normalized_word_Total(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * (len(w1) + len(w2))



df['word_Total'] = df.apply(normalized_word_Total, axis=1)

print "\n-> word_Total =(Total num of words in Question 1 + Total num of words in Question 2)"

def normalized_word_share(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * len(w1 & w2)/(len(w1) + len(w2))



df['word_share'] = df.apply(normalized_word_share, axis=1)



-> word_Total =(Total num of words in Question 1 + Total num of words in Question 2)


In [47]:
#def bigram_word_Common(row):
#    w1 = set(nltk.bigrams(row['question1'].split(" ")).split())
#    w2 = set(nltk.bigrams(row['question2'].split(" ")).split())
#    return 1.0 * len(w1 & w2)

#df['bigram_common'] = df.apply(bigram_word_Common, axis=1)

df['freq_qid1'] = df.groupby('qid1')['qid1'].transform('count') 

print "-> freq_qid1 = frequency of qid1's"

df['freq_qid2'] = df.groupby('qid2')['qid2'].transform('count')

print "\n-> freq_qid2 = frequency of qid2's "

df['q1len'] = df['question1'].str.len() 

print "\n-> q1len = Length of q1"

df['q2len'] = df['question2'].str.len()

print "\n-> q2len = Length of q2"

df['q1_n_words'] = df['question1'].apply(lambda row: len(row.split(" ")))

print "\n-> q1_n_words = num of words i Question 1"
# how lambda is workiing
df['q2_n_words'] = df['question2'].apply(lambda row: len(row.split(" ")))

print "\n-> q2_n_words = num of words i Question 2"



def normalized_word_Common(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * len(w1 & w2)




df['word_Common'] = df.apply(normalized_word_Common, axis=1)

print "\n-> word_Common = (num of common unique words in Question 1 and Question 2)"

def normalized_word_Total(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * (len(w1) + len(w2))



df['word_Total'] = df.apply(normalized_word_Total, axis=1)

print "\n-> word_Total =(Total num of words in Question 1 + Total num of words in Question 2)"

def normalized_word_share(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
    return 1.0 * len(w1 & w2)/(len(w1) + len(w2))



df['word_share'] = df.apply(normalized_word_share, axis=1)

print "\n-> word_share = (num of common unique words in Question 1 and Question 2)/(Total num of words in Question 1 + Total num of words in Question 2)\n"

print "\n-> freq_q1+q2 = sum total of frequency of qid1 and qid2 "

print "\n-> freq_q1-q2 = absolute difference of frequency of qid1 and qid2 "

df['freq_q1+q2'] = df['freq_qid1']+df['freq_qid2']
df['freq_q1-q2'] = abs(df['freq_qid1']-df['freq_qid2'])

#Bigrams
def bigram_word_Common(row):
    w1 = set(nltk.bigrams(row['question1'].split(" ")))
    w2 = set(nltk.bigrams(row['question2'].split(" ")))
    return 1.0 * len(w1 & w2)

df['bigram_common'] = df.apply(bigram_word_Common,axis = 1)

def bigram_word_total(row):
    w1 = set(nltk.bigrams(row['question1'].split(" ")))
    w2 = set(nltk.bigrams(row['question2'].split(" ")))
    return 1.0 * ((len(w1)+len(w2)) - 1.0 * len(w1 & w2))

df['bigram_total'] = df.apply(bigram_word_total,axis = 1)

df['bigram_ratio'] = df['bigram_common']/df['bigram_total']


df.to_csv("df_fe2_preprocessing_train.csv", index=False)

df.head()


-> freq_qid1 = frequency of qid1's

-> freq_qid2 = frequency of qid2's 

-> q1len = Length of q1

-> q2len = Length of q2

-> q1_n_words = num of words i Question 1

-> q2_n_words = num of words i Question 2

-> word_Common = (num of common unique words in Question 1 and Question 2)

-> word_Total =(Total num of words in Question 1 + Total num of words in Question 2)

-> word_share = (num of common unique words in Question 1 and Question 2)/(Total num of words in Question 1 + Total num of words in Question 2)


-> freq_q1+q2 = sum total of frequency of qid1 and qid2 

-> freq_q1-q2 = absolute difference of frequency of qid1 and qid2 


,id,qid1,qid2,question1,question2,is_duplicate,bigram_common,bigram_total,bigram_ratio,combined,...,q1len,q2len,q1_n_words,q2_n_words,freq_q1+q2,freq_q1-q2,capital_1,capital_2,capital_total,capital_diff
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,11.0,15.0,0.733333,what is the step by step guide to invest in sh...,...,66,57,15,13,2,0,1.0,0.0,0.0,0.0
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,6.0,23.0,0.260870,what is the story of kohinoor koh i noor dia...,...,51,88,13,18,5,3,5.0,0.0,0.0,0.0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,1.0,23.0,0.043478,how can i increase the speed of my internet co...,...,73,59,15,11,2,0,5.0,0.0,0.0,0.0
3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math math is di...,0,0.0,23.0,0.000000,why am i mentally very lonely how can i solve...,...,50,57,13,19,2,0,4.0,0.0,0.0,0.0
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.0,22.0,0.000000,which one dissolve in water quikly sugar salt...,...,76,39,16,8,4,2,1.0,0.0,0.0,0.0


In [42]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,bigram_common,bigram_total,bigram_ratio,combined,...,q1len,q2len,q1_n_words,q2_n_words,freq_q1+q2,freq_q1-q2,capital_1,capital_2,capital_total,capital_diff
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,11.0,15.0,0.733333,what is the step by step guide to invest in sh...,...,66,57,15,13,2,0,0.0,0.0,0.0,0.0
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,6.0,23.0,0.260870,what is the story of kohinoor koh i noor dia...,...,51,88,13,18,5,3,0.0,0.0,0.0,0.0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,1.0,23.0,0.043478,how can i increase the speed of my internet co...,...,73,59,15,11,2,0,0.0,0.0,0.0,0.0
3,3,7,8,why am i mentally very lonely how can i solve...,find the remainder when math math is di...,0,0.0,23.0,0.000000,why am i mentally very lonely how can i solve...,...,50,57,13,19,2,0,0.0,0.0,0.0,0.0
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.0,22.0,0.000000,which one dissolve in water quikly sugar salt...,...,76,39,16,8,4,2,0.0,0.0,0.0,0.0


In [ ]:
def get_token_features(q1, q2):
    token_features = [0.0]*10
    
    # Converting the Sentence of Que. in the Tokens
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features
    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    # Last word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    # First word of both question is same or not
    token_features[8] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    token_features[9] = (len(q1_tokens) + len(q2_tokens))/2
    return token_features


In [ ]:
# get the Longest Common sub string

def get_longest_substr_ratio(a, b):
    strs = list(distance.lcsubstrings(a, b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b)) + 1)

In [ ]:
def extract_features(df):
    df["question1"] = df["question1"].fillna("").apply(preprocess)
    df["question2"] = df["question2"].fillna("").apply(preprocess)

    print("token features...")
    
    # Merging Features with dataset
    
    token_features = df.apply(lambda x: get_token_features(x["question1"], x["question2"]), axis=1)
    
    df["cwc_min"]       = list(map(lambda x: x[0], token_features))
    df["cwc_max"]       = list(map(lambda x: x[1], token_features))
    df["csc_min"]       = list(map(lambda x: x[2], token_features))
    df["csc_max"]       = list(map(lambda x: x[3], token_features))
    df["ctc_min"]       = list(map(lambda x: x[4], token_features))
    df["ctc_max"]       = list(map(lambda x: x[5], token_features))
    df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
    df["first_word_eq"] = list(map(lambda x: x[7], token_features))
    df["abs_len_diff"]  = list(map(lambda x: x[8], token_features))
    df["mean_len"]      = list(map(lambda x: x[9], token_features))
   
    #Computing Fuzzy Features and Merging with Dataset
    print("fuzzy features..")
    df["token_set_ratio"]       = df.apply(lambda x: fuzz.token_set_ratio(x["question1"], x["question2"]), axis=1)
    df["token_sort_ratio"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["question1"], x["question2"]), axis=1)
    df["fuzz_ratio"]            = df.apply(lambda x: fuzz.QRatio(x["question1"], x["question2"]), axis=1)
    df["fuzz_partial_ratio"]    = df.apply(lambda x: fuzz.partial_ratio(x["question1"], x["question2"]), axis=1)
    df["longest_substr_ratio"]  = df.apply(lambda x: get_longest_substr_ratio(x["question1"], x["question2"]), axis=1)
    return df



In [ ]:
print("Extracting features for train:")
train_df = pd.read_csv("train.csv")
train_df = extract_features(train_df)
#train_df.drop(["id", "qid1", "qid2", "question1", "question2", "is_duplicate"], axis=1, inplace=True)

# Creating new .csv File with total 21 features:
train_df.to_csv("nlp_features_train.csv", index=False)